# Lesson6 ニューラルネットに画像を生成させよう

## 目次

- Section5 参考
  - 5.1 VAE

### 5.1 VAE

今回は深層生成モデルのGANの実装をメインに扱いましたが、深層生成モデルはGANだけではありません。

興味のある方は、その例として Variational Auto Encoder (VAE)というモデルを使ってニューラルネットに画像生成に取り組んでみましょう。

### 5.1.1 VAEの概要

VAEではデータの潜在的な意味を表す潜在変数(z)からデータを生成する確率分布( $p_θ(x|z)$ ) を求めます。

潜在的な意味とは、例えばMNISTの手書き文字であれば、文字であることや筆跡などがあたります。
以下ではMNISTを例に話を進めていきます。


下の図のようにMNISTのデータセットの入力の「1」という手書き文字の画像を潜在変数 **z**に変換し、確率分布　$p_θ(x|z)$　によって入力画像を再現するネットワークを考えます。

このとき、入力を潜在変数に変換するネットワークのことを **Encoder**、潜在変数から出力として入力画像を再現するネットワークのことを **Decoder**　といいます。

確率分布　$p_θ(x|z)$　を求めることによって任意の潜在変数から手書き文字画像と似た画像を出力出来るようになります。

<img src='../figures/vae1_var2.png' alt='vae_1'>

Encoderでは入力の画像から潜在変数zをサンプルする必要があり、VAEでは潜在変数のガウス分布性を仮定します。

そのため、以下のように入力からサンプルするためのパラメータとしてガウス分布の平均と分散を出力することを行います。

<img src='../figures/vae2_var2.png' alt='vae_2'>

以上をふまえて、作成するモデルは以下のようなモデルを構築することになります。

<img src='../figures/vae_3.png' alt='vae_3'>

### 5.1.2 Encoder

今回の実装ではEncoderにあたる層は1層のみです。

### 補足

今回はEncoderの出力として分散のlogとして出力を行っていますが、これは分散の値域を正にするためです。分散は $\log(z\_log\_var)$とすることで得られます。
また、関数の出力が正となるsoftplus関数を使用して分散も表現することも可能です。
softplus関数については[こちら](http://tecmemo.wpblog.jp/2017/01/08/dl_activation-function/)をご参照下さい。

### 5.1.3 潜在変数

Encoderで出力した平均μと分散σから潜在変数zをサンプリングします。

### 5.1.4 Decoder

潜在変数zから画像を生成するDecoderを定義します。

### 5.1.5 Loss

周辺尤度の変分下界の最大化によって最適化を行います。

Lossとしては以下に示す復元誤差と正則化項の和になります。
(なお、デコーダにベルヌーイ分布を仮定するため復元誤差は負の交差エントロピー誤差となります。）

#### 復元誤差

$ \sum_{k=1}^{n} ( x_k \log(y_k) + (1-x_k) \log(1-y_k) ) $

#### 正則化項

$-\frac{1}{2}\sum_{k=1}^{n} ( 1 + \log(\sigma_i^2) - \mu_i^2 - \sigma_i^2 )$

ここでLossを定義するために自作のLayerを用意します。

KerasのLayer classを継承したクラスでcallメソッドの中で `add_loss`というメソッドを用いてlossを定義することが出来ます。

### 補足: 正則化項の意味

#### 1. VAEと多様体学習

VAEは多様体学習を行うことを目的としています。例えば、以下のように2次元の空間上にデータを表現出来るようにすることが目的となります。正則化項の制限によってこの多様体の学習がうまく出来るようになります。

<img src='../figures/vae_suppliment.png' alt='vae_suppliment'>

( 引用: [Auto-Encoding Variational Bayes](https://arxiv.org/abs/1312.6114) )


#### 2. 正則化項の役割

ただし、正則化項が強く作用すると画像の特徴の情報が小さくなるのも確かで、正則化項を段々と強くかける（正則化項に係数をかけて、その係数を学習が進むにつれて0から増やしていく）といった学習方法もあります。
また、その係数を1よりもあえて大きく取ることで、disentangleな情報を表現出来ることも知られています(β-VAE)。disentangleな情報が出来るとは例えば以下のように「方向」、「幅」、「脚のスタイル」を多様体学習した潜在変数の軸ごとに獲得出来るということです。

<img src='../figures/vae_suppliment2.png' alt='vae_suppliment2'>

( 引用: [β-VAE: LEARNING BASIC VISUAL CONCEPTS WITH A CONSTRAINED VARIATIONAL FRAMEWORK](https://openreview.net/pdf?id=Sy2fzU9gl))

### 5.1.6 MNISTによる学習

MNISTのデータセットによって学習を行います。

### 5.1.7 手書き文字の生成

任意の潜在変数によって手書き文字が出力出来ることを実際に画像を生成して確認してみましょう。